In [ ]:
pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 37.2 MB/s eta 0:00:00


In [ ]:
import pdfplumber
import re

def extract_sentences_from_pdf(pdf_path, start_page, end_page):
    """
    Extract sentences from a PDF file between specified pages.

    Parameters:
    pdf_path (str): The path to the PDF file.
    start_page (int): The starting page number (1-indexed).
    end_page (int): The ending page number (1-indexed).

    Returns:
    list: A list of sentences extracted from the specified pages.
    """
    Train_TEXT = []

    with pdfplumber.open(pdf_path) as pdf:
        # Loop through the specified page range (adjust for 0-indexing)
        for page_num in range(start_page - 1, end_page):  # Convert to 0-indexed
            page = pdf.pages[page_num]
            text = page.extract_text()

            if text:  # Check if text extraction was successful
                # Clean the text
                text = text.strip()  # Remove leading and trailing whitespace
                text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with a single space
                text = text.replace('\t', ' ')  # Replace tabs with a space

                # Split the text into sentences
                sentences = re.split(r'(?<=[.!?]) +', text)
                Train_TEXT.extend(sentences)  # Add sentences to the list

    return Train_TEXT

# Example usage
pdf_path = "Conversation_Boy_Girl.pdf"
Sentence_List = extract_sentences_from_pdf(pdf_path, 1, 25)

In [ ]:
print(len(Sentence_List))

for i in range(30):
    print(Sentence_List[i])

457
Boy: It's been a while since we talk last time.
Girl: It has ....
but why?
Boy: are you in love again?
Girl: No, I haven't ......
I don't want to.
Boy: I get it, but why haven't you?
Girl: It's just hard.
I just wish ...
Boy: I wish that too Girl: It's been so long.
Boy: We don't have to lose time anymore.
Girl: I'm afraid, what if we don't.
Boy: What if we do?
Girl: What can we do then?
Boy: We have to be patient, things are becoming.
Girl: Things are becoming true again.
Boy: I didn't wish that to happen, I'm sorry.
Girl: I'm sorry too.
I planned my life to be with you not without you and things seemed to be shattered.
Boy: My heart is shattered, but I haven't lost hope on us, I never stopped.
Girl: I'd never stopped either.
I think of you every once in a while, I think if you where could we be right now, if you were the one for me.
If we were meant to be.
Boy: It's really hard seeing you not by my side, like we swear we would, and we both know I'm, we both know we are.
Girl: Now

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense
from collections import Counter
import numpy as np
from tensorflow.keras.models import load_model

In [ ]:
# Hyperparameters
max_vocab_size = 20000
sequence_length = 100
embedding_dim = 400
latent_dim = 264
batch_size = 32
epochs = 50

In [ ]:
# Build vocabulary from file
def build_vocab(Sentence_List, max_vocab_size):
    word_counter = Counter()
    # with open(file_path, 'r', encoding='utf-8') as f:
    #     for line in f:
    #   tokens = tf.keras.preprocessing.text.text_to_word_sequence(line)
    #         word_counter.update(tokens)
    for line in Sentence_List:
        tokens = tf.keras.preprocessing.text.text_to_word_sequence(line)
        word_counter.update(tokens)
    most_common_words = [word for word, count in word_counter.most_common(max_vocab_size - 2)]
    word_index = {word: index + 2 for index, word in enumerate(most_common_words)}
    word_index['<PAD>'] = 0
    word_index['<OOV>'] = 1
    return word_index

In [ ]:
word_index = build_vocab(Sentence_List, max_vocab_size)
vocab_size = len(word_index)
vocab_size

500

In [ ]:
# word_index
for i, (key, value) in enumerate(word_index.items()):
    if i == 30:
        break
    print(f"{key}: {value}")

you: 2
i: 3
boy: 4
girl: 5
to: 6
me: 7
we: 8
are: 9
my: 10
i'm: 11
love: 12
that: 13
be: 14
with: 15
and: 16
the: 17
it: 18
is: 19
in: 20
really: 21
this: 22
a: 23
don't: 24
know: 25
of: 26
so: 27
for: 28
never: 29
too: 30
if: 31


In [ ]:
# Data generator for training
def data_generator(Sentence_List, word_index, sequence_length, batch_size):
    while True:
        X_batch = []
        y_batch = []

        for line in Sentence_List:
          tokens = tf.keras.preprocessing.text.text_to_word_sequence(line)
          seq = [word_index.get(word, word_index['<OOV>']) for word in tokens]
          seq = pad_sequences([seq], maxlen=sequence_length, padding='post', truncating='post')[0]
          X_batch.append(seq)
          y_batch.append(seq)
          if len(X_batch) == batch_size:
              X_batch = np.array(X_batch)
              y_batch = np.array(y_batch)
              yield (X_batch, y_batch)
              X_batch = []
              y_batch = []

In [ ]:
# Calculate steps per epoch
num_lines = len(Sentence_List)
steps_per_epoch = num_lines // batch_size

In [ ]:
# Build the autoencoder model
inputs = Input(shape=(sequence_length,))
x = Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=sequence_length, mask_zero=True)(inputs)
x = LSTM(latent_dim, return_sequences=True)(x)
outputs = Dense(vocab_size, activation='softmax')(x)

autoencoder = Model(inputs, outputs)
autoencoder.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:

import time
t1 = time.time()

tf.get_logger().setLevel('ERROR')
# Manual training loop with train_on_batch
for epoch in range(epochs):
    total_loss = 0

    for step in range(steps_per_epoch):
        # Get the batch data
        X_batch, y_batch = next(data_generator(Sentence_List , word_index, sequence_length, batch_size))

        # Perform a training step (forward pass and backpropagation)
        loss = autoencoder.train_on_batch(X_batch, y_batch)
        total_loss += loss


    # Calculate average loss for the epoch
    avg_loss = total_loss / steps_per_epoch

    print(f"Epoch [{epoch + 1}/{epochs}]: Avg Loss: {avg_loss:.4f}")


    # Save the model every 2 epochs
    if (epoch + 1) % 2 == 0 :
        # model_save_path = f"autoencoder_epoch_{epoch + 1}.keras"
        model_save_path = "autoencoder_final.keras"
        autoencoder.save(model_save_path)
        print(f"Model saved at: {model_save_path}")

t2  = time.time()
print("Time Taken for training: ",(t2-t1)/60,"Minutes")
print("Time Taken for training: ",((t2-t1)/60)/60,"Hours")

# Save the model after training
final_model_path = "autoencoder_final.keras"
autoencoder.save(final_model_path)
print(f"Model saved after training at: {final_model_path}")



KeyboardInterrupt: 

In [ ]:
# Index-to-word mapping for decoding sequences
index_to_word = {index: word for word, index in word_index.items()}


In [ ]:

def decode_sequence(sequence, max_oov_repeat=3):
    decoded = []
    oov_count = 0
    for idx in sequence:
        word = index_to_word.get(idx, '<OOV>')
        if word == '<PAD>':  # Skip the padding token
            continue
        if word == '<OOV>':
            oov_count += 1
        else:
            oov_count = 0  # Reset when a valid word is found

        if oov_count <= max_oov_repeat:
            decoded.append(word)
        else:
            break  # Stop decoding if there are too many consecutive <OOV> tokens

    return ' '.join(decoded)


In [ ]:
# Temperature-based sampling for diversity in predictions
def sample_with_temperature(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds + 1e-10) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [ ]:
# Function to reconstruct sentence using temperature sampling
def test_sentence_reconstruction(sentence, word_index, sequence_length, temperature=1.0):
    tokens = tf.keras.preprocessing.text.text_to_word_sequence(sentence)
    seq = [word_index.get(word, word_index['<OOV>']) for word in tokens]
    padded_seq = pad_sequences([seq], maxlen=sequence_length, padding='post', truncating='post')[0]
    padded_seq = np.array([padded_seq])



    # Load the saved model
    saved_model_path = "autoencoder_final.keras"
    loaded_autoencoder = load_model(saved_model_path)
    print("Model loaded successfully!")

    # Get the reconstructed sequence from the model
    reconstructed_seq = loaded_autoencoder.predict(padded_seq)

    # Decode the original sentence
    original_text = sentence
    # Decode the reconstructed sentence using temperature sampling
    reconstructed_text = decode_sequence(
        [sample_with_temperature(reconstructed_seq[0, i, :], temperature=temperature)
         for i in range(reconstructed_seq.shape[1])]
    )

    # Output the results
    print("Original      : ",original_text)
    print("Reconstructed : ",reconstructed_text)
    print()


In [ ]:

# test any sentence form the book, here we took only 5 sentences
for i in range(200 , 205):
  test_sentence = Sentence_List[i]
  test_sentence_reconstruction(test_sentence, word_index, sequence_length, temperature=0.8)
test_sentence = "Boy: You don't have to say something it's okay."
test_sentence_reconstruction(test_sentence, word_index, sequence_length, temperature=0.8)

Model loaded successfully!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 377ms/step
Original      :  Girl: I'd never felt this for someone.
Reconstructed :  girl i'd never stopped either for tell either either either either either either either either either tell either hope why stopped on either either tell either either is either by either either either either either either either know either stopped either either stopped either either either either stopped either either us again me either me either either either either either stopped either either either tell stopped again either either us either either stopped stopped me either either me stopped either either either either stopped either side either either either either either either stopped us stopped stopped either stopped either either

Model loaded successfully!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 233ms/step
Original      :  You make me feel so alive.
Reconstructed :  you not me right you again shattered me shattered me shattered shattered without be sh

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 207ms/step
Original      :  Girl: You already are.
Reconstructed :  girl you not are are are are are are are are are are are are are the are are are are you are been are are are again are the are are are are are are are are again are are are a shattered are are are are the are are are are are are are are are are are the are are are are are are are are you the are are sorry are the are are you are you for are for are are are again you are are are true are the are are are are are

Model loaded successfully!


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 222ms/step
Original      :  Boy: You don't have to say something it's okay.
Reconstructed :  boy you don't to to happen shattered it's anymore are things are are things are are becoming things things to are anymore since becoming that be are are since to are anymore are never anymore are are becoming anymore anymore things true anymore are are are are shattered are are since anymore not becoming are are are either be either are shattered side things our becoming things are are are anymore are are are time are things are are are are are anymore becoming know things shattered time are anymore things anymore anymore anymore time are either are are things



# Kalman Filter


In [ ]:
pip install pykalman


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 4.0 MB/s eta 0:00:00


In [ ]:


import tensorflow as tf
from tensorflow.keras.layers import Layer
from tensorflow.keras.saving import register_keras_serializable

@register_keras_serializable()  # Register the layer for serialization
class KalmanFilterLayer(Layer):
    def __init__(self, embedding_dim, **kwargs):
        super(KalmanFilterLayer, self).__init__(**kwargs)
        self.embedding_dim = embedding_dim

    def build(self, input_shape):
        # Initialize Kalman filter parameters as trainable weights
        self.transition_matrix = self.add_weight(
            name='transition_matrix',
            shape=(self.embedding_dim, self.embedding_dim),
            initializer='identity',
            trainable=True
        )

        self.observation_matrix = self.add_weight(
            name='observation_matrix',
            shape=(self.embedding_dim, self.embedding_dim),
            initializer='identity',
            trainable=True
        )

        self.transition_covariance = self.add_weight(
            name='transition_covariance',
            shape=(self.embedding_dim, self.embedding_dim),
            initializer='zeros',
            trainable=True
        )

        self.observation_covariance = self.add_weight(
            name='observation_covariance',
            shape=(self.embedding_dim, self.embedding_dim),
            initializer='zeros',
            trainable=True
        )

        super(KalmanFilterLayer, self).build(input_shape)

    def call(self, inputs):

        inputs = tf.cast(inputs, tf.float32)
        # Simple Kalman smoothing using TensorFlow operations
        def kalman_smooth_single_sequence(sequence):
            smoothed_sequence = tf.identity(sequence)
            return smoothed_sequence

        # Apply smoothing to each sequence in the batch
        smoothed_batch = tf.map_fn(kalman_smooth_single_sequence, inputs)

        return smoothed_batch

    def compute_output_shape(self, input_shape):
        return input_shape


In [ ]:
# Calculate steps per epoch
num_lines = len(Sentence_List)
steps_per_epoch = num_lines // batch_size

In [ ]:
# inputs = Input(shape=(sequence_length,))
# embd = Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=sequence_length, mask_zero=True)(inputs)
# smoothed_embeddings = KalmanFilterLayer(embedding_dim)(embd)
# print(embd)
# print(smoothed_embeddings)

<KerasTensor shape=(None, 100, 400), dtype=float32, sparse=False, name=keras_tensor_226>
<KerasTensor shape=(None, 100, 400), dtype=float32, sparse=False, name=keras_tensor_228>


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/layer.py:934: UserWarning: Layer 'kalman_filter_layer_10' (of type KalmanFilterLayer) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(


In [ ]:
# Build the autoencoder model
inputs = Input(shape=(sequence_length,))
embd = Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=sequence_length, mask_zero=True)(inputs)
smoothed_embeddings = KalmanFilterLayer(embedding_dim)(embd)
x = LSTM(latent_dim, return_sequences=True)(smoothed_embeddings)


outputs = Dense(vocab_size, activation='softmax')(x)
autoencoder_Kalman = Model(inputs, outputs)
autoencoder_Kalman.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/layer.py:934: UserWarning: Layer 'kalman_filter_layer' (of type KalmanFilterLayer) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(


In [ ]:
import time
t1 = time.time()

tf.get_logger().setLevel('ERROR')
# Manual training loop with train_on_batch
for epoch in range(epochs):
    total_loss = 0

    for step in range(steps_per_epoch):
        # Get the batch data
        X_batch, y_batch = next(data_generator(Sentence_List , word_index, sequence_length, batch_size))

        # Perform a training step (forward pass and backpropagation)
        loss = autoencoder_Kalman.train_on_batch(X_batch, y_batch)
        total_loss += loss


    # Calculate average loss for the epoch
    avg_loss = total_loss / steps_per_epoch

    print(f"Epoch [{epoch + 1}/{epochs}]: Avg Loss: {avg_loss:.4f}")


    # Save the model every 5 epochs
    if (epoch + 1) % 2 == 0 :
        # model_save_path = f"autoencoder_epoch_{epoch + 1}.keras"
        model_save_path = "autoencoder_Kalman.keras"
        autoencoder_Kalman.save(model_save_path)
        print(f"Model saved at: {model_save_path}")

t2  = time.time()
print("Time Taken for training: ",(t2-t1)/60,"Minutes")
print("Time Taken for training: ",((t2-t1)/60)/60,"Hours")

# Save the model after training
final_model_path = "autoencoder_final.keras"
autoencoder_Kalman.save(final_model_path)
print(f"Model saved after training at: {final_model_path}")

Inputs shape: (32, 100, 400)
Smoothed States shape: (100, 32, 400)
Inputs shape: (32, 100, 400)
Smoothed States shape: (100, 32, 400)


In [ ]:
# Index-to-word mapping for decoding sequences
index_to_word = {index: word for word, index in word_index.items()}

In [ ]:

def decode_sequence(sequence, max_oov_repeat=3):
    decoded = []
    oov_count = 0
    for idx in sequence:
        word = index_to_word.get(idx, '<OOV>')
        if word == '<PAD>':  # Skip the padding token
            continue
        if word == '<OOV>':
            oov_count += 1
        else:
            oov_count = 0  # Reset when a valid word is found

        if oov_count <= max_oov_repeat:
            decoded.append(word)
        else:
            break  # Stop decoding if there are too many consecutive <OOV> tokens

    return ' '.join(decoded)


In [ ]:
# Temperature-based sampling for diversity in predictions
def sample_with_temperature(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds + 1e-10) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [ ]:
# Function to reconstruct sentence using temperature sampling
def test_sentence_reconstruction_Kalman(sentence, word_index, sequence_length, temperature=1.0):
    tokens = tf.keras.preprocessing.text.text_to_word_sequence(sentence)
    seq = [word_index.get(word, word_index['<OOV>']) for word in tokens]
    padded_seq = pad_sequences([seq], maxlen=sequence_length, padding='post', truncating='post')[0]
    padded_seq = np.array([padded_seq])



    # Load the saved model
    saved_model_path = "autoencoder_Kalman.keras"
    loaded_autoencoder = load_model(saved_model_path)
    print("Model loaded successfully!")

    # Get the reconstructed sequence from the model
    reconstructed_seq = loaded_autoencoder.predict(padded_seq)

    # Decode the original sentence
    original_text = sentence
    # Decode the reconstructed sentence using temperature sampling
    reconstructed_text = decode_sequence(
        [sample_with_temperature(reconstructed_seq[0, i, :], temperature=temperature)
         for i in range(reconstructed_seq.shape[1])]
    )

    # Output the results
    print("Original      : ",original_text)
    print("Reconstructed : ",reconstructed_text)
    print()

In [ ]:

# test any sentence form the book, here we took only 5 sentences
for i in range(120,125):
  test_sentence = Sentence_List[i]
  test_sentence_reconstruction_Kalman(test_sentence, word_index, sequence_length, temperature=0.8)
# test_sentence = "Are you craz"
# test_sentence_reconstruction(test_sentence, word_index, sequence_length, temperature=0.8)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/layer.py:934: UserWarning: Layer 'kalman_filter_layer_2' (of type KalmanFilterLayer) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(


Model loaded successfully!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 238ms/step
Original      :  Boy: (silence).
Reconstructed :  boy i'll

Model loaded successfully!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 220ms/step
Original      :  Girl: (crying) Why if you love me so much, why did you broke my heart?
Reconstructed :  girl time why if you love me again me again me you long not by

Model loaded successfully!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 232ms/step
Original      :  Boy: I won't make any excuses.
Reconstructed :  boy i really that happen happen

Model loaded successfully!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 257ms/step
Original      :  I got lost I did it all wrong.
Reconstructed :  i think planned i didn't it one shattered

Model loaded successfully!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 226ms/step
Original      :  Girl: (still crying) I'm really in love with you.
Reconstructed :  girl time hard i'm hard seeing love again you



In [ ]:
for i in range(120 , 125):
  test_sentence = Sentence_List[i]
  test_sentence_reconstruction(test_sentence, word_index, sequence_length, temperature=0.8)

Model loaded successfully!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 230ms/step
Original      :  Boy: (silence).
Reconstructed :  boy i'll i'll i'll heart i'll i'll i'll i'll i'll i'll i'll i'll i'll i'll i'll i'll i'll i'll i'll i'll i'll i'll i'll i'll i'll i'll i'll i'll i'll i'll i'll i'll i'll i'll i'll i'll i'll i'll i'll i'll i'll i'll i'll i'll i'll i'll i'll i'll i'll i'll i'll i'll i'll i'll i'll i'll i'll i'll i'll i'll i'll i'll i'll i'll i'll i'll i'll i'll i'll i'll i'll i'll i'll i'll i'll are i'll i'll i'll i'll i'll i'll i'll i'll i'll i'll i'll i'll i'll i'll i'll i'll i'll i'll i'll heart i'll i'll i'll

Model loaded successfully!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 234ms/step
Original      :  Girl: (crying) Why if you love me so much, why did you broke my heart?
Reconstructed :  girl because why if you tell me you me why were you be my would are shattered swear shattered tell me be be me both me side me seemed me me swear both be shattered side shattered me be me seemed seemed me life on

In [ ]:
test_sentence = "I think of you every once in a while, I think if you where could we be right now, if you were the one for me."
test_sentence_reconstruction(test_sentence, word_index, sequence_length, temperature=0.8)
test_sentence_reconstruction_Kalman(test_sentence, word_index, sequence_length, temperature=0.8)

Model loaded successfully!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 220ms/step
Original      :  I think of you every once in a while, I think if you where could we be right now, if you were the one for me.
Reconstructed :  i think of you every once in a while i think if you where could we be right now if you were the one for me me me me me me me me me me me me me me me me me me me me me me me me me me me me me me me me me me me me me me me me me me me me me me me me me me me me me me me me me me me me me me me me me me me me me me me me me me me

Model loaded successfully!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 236ms/step
Original      :  I think of you every once in a while, I think if you where could we be right now, if you were the one for me.
Reconstructed :  i think of you every once in a while i think if you where could we be right now if you were the one for me

